# Sampling from a diffusion model
In this notebooks we will sampled from the previously trained diffusion model.
- We are going to compare the samples from DDPM and DDIM samplers
- Visualize mixing samples with conditional diffusion models

In [ ]:
from types import SimpleNamespace
from pathlib import Path
import torch
import torch.nn.functional as F
import numpy as np
from utilities import *

import wandb

In [ ]:
wandb.login(anonymous="allow")

# Setting Things Up

In [ ]:
# Wandb Params
PROJECT = "dlai_sprite_diffusion"
MODEL_ARTIFACT = "deeplearning-ai-temp/model-registry/SpriteGen:latest" 

# ddpm sampler hyperparameters
timesteps = 500
beta1 = 1e-4
beta2 = 0.02
num_samples = 30
height = 16
ddim_n = 25

device = get_device()

# we are storing the parameters in a dictionary to be logged to wandb
config = SimpleNamespace(
    timesteps=timesteps,
    beta1=beta1,
    beta2=beta2,
    num_samples=num_samples,
    height=height,
    ddim_n=ddim_n,
    device=device,
)

In the previous notebook we saved the best model as a wandb Artifact (our way of storing files during runs). We will now load the model from wandb and set up the sampling loop.

In [ ]:
def load_model(model_artifact_name):
    "Load the model from wandb artifacts"
    api = wandb.Api()
    artifact = api.artifact(model_artifact_name, type="model")
    model_path = Path(artifact.download())

    # recover model info from the registry
    producer_run = artifact.logged_by()

    # load the weights dictionary
    model_weights = torch.load(model_path/"context_model.pth", 
                               map_location="cpu")

    # create the model
    model = ContextUnet(in_channels=3, 
                        n_feat=producer_run.config["n_feat"], 
                        n_cfeat=producer_run.config["n_cfeat"], 
                        height=producer_run.config["height"])
    
    # load the weights into the model
    model.load_state_dict(model_weights)

    # set the model to eval mode
    model.eval()
    return model.to(device)

In [ ]:
nn_model = load_model(MODEL_ARTIFACT)

## Sampling

We will sample and log the generated samples to wandb.

In [ ]:
_, sample_ddpm_context = setup_ddpm(beta1, beta2, timesteps, device)

Let's define a set of noises and a context vector to condition on.

In [ ]:
# Noise vector
# x_T ~ N(0, 1), sample initial noise
noises = torch.randn(num_samples, 3, height, height).to(device)  

# A fixed context vector to sample from
ctx_vector = F.one_hot(torch.tensor([0,0,0,0,0,0,
                                     1,1,1,1,1,1,
                                     2,2,2,2,2,2,
                                     3,3,3,3,3,3,
                                     4,4,4,4,4,4]), 
                       5).to(device=device).float()

Let's bring that faster DDIM sampler from the diffusion course.

In [ ]:
sample_ddim_context = setup_ddim(beta1, beta2, timesteps, device)

Let's create a `wandb.Table` to store our generations

In [ ]:
table = wandb.Table(columns=["input_noise", "ddpm", "ddim", "class"])

let's compute ddpm samples as before

In [ ]:
ddpm_samples, _ = sample_ddpm_context(nn_model, noises, ctx_vector)

For DDIM we can control the step size by the `n` param:

In [ ]:
ddim_samples, _ = sample_ddim_context(nn_model, noises, ctx_vector, n=ddim_n)

Let's put the class names as a column on the table

In [ ]:
def ctx_to_classes(ctx_vector):
    classes = "hero,non-hero,food,spell,side-facing".split(",")
    return [classes[i] for i in ctx_vector.argmax(dim=1)]

Let's keep track of the sampling params on a dictionary

We can add the rows to the table one by one, we also cast images to `wandb.Image` so we can render them correctly in the UI

In [ ]:
for noise, ddpm_sample, ddim_sample, c in zip(noises, ddpm_samples, ddim_samples, ctx_to_classes(ctx_vector)):
    table.add_data(wandb.Image(noise), 
                   wandb.Image(ddpm_sample), 
                   wandb.Image(ddim_sample),
                   c)

we log the table to W&B, we can also use `wandb.init` as a context manager, this way we ensure that the run is finished when exiting the manager.

In [ ]:
with wandb.init(project=PROJECT, job_type="samplers_battle", anonymous="allow", config=config):
    wandb.log({"samplers_table":table})